In [1]:
! pip install -q --upgrade pip
! pip install -q transformers[torch]
! pip install -q datasets==2.17.0 peft

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.5 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.4.0 which

In [2]:
class Config(): 
    save_path = "dataset_for_finetune_pygmalion"
    username = 'doublecringe123'
    
cfg = Config()

import os
from kaggle_secrets import UserSecretsClient

os.environ["HF_TOKEN"] = UserSecretsClient().get_secret("HF_TOKEN")

def v_check(): 
    import datasets 
    import transformers
    return datasets.__version__, transformers.__version__

v_check()

('2.17.0', '4.38.2')

In [3]:
import os
os.environ["MODEL_DIR"] = 'models'

# Define NewDataset class

In [4]:
! wget -O "mds.py" "https://raw.githubusercontent.com/goin2crazy/multy-dataset/main/main.py" 

from mds import NewDataset

--2024-04-15 14:30:07--  https://raw.githubusercontent.com/goin2crazy/multy-dataset/main/main.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2333 (2.3K) [text/plain]
Saving to: 'mds.py'

mds.py              100%[===================>]   2.28K  --.-KB/s    in 0s      

2024-04-15 14:30:08 (39.1 MB/s) - 'mds.py' saved [2333/2333]



# Load Datasets, Preprocess

In [5]:
# rename #person*# to 'You' or '[CHARACTER]'
def preprocess_summary(text):
  return text.replace("#Person1#", "You").replace("#Person2#", "[CHARACTER]")

def split_dialogue(dataset):
  # split all dialogue text to messages
  def preprocess_dialogue(text):
    # replace persona 1 name one to be main character
    d = (text
        .replace("#Person1#:", "#character1#")
        .replace("#Person2#:", "#character2#")
        .split("#character"))

    # clear from empty items
    d = [t for t in d if len(t) > 0]
    # define which text is (user or assistant)
    d = [{'content': preprocess_summary(text[2:].strip()), 'role': 'user' if (text.startswith('1#')) else 'assistant'} for text in d]
    return d

  return dataset.map(lambda x: {'d': preprocess_dialogue(x['dialogue']), 's': preprocess_summary(x['summary'])})

def preprocess_empathetic_dialogues_llm(dataset):
  from datasets import DatasetDict

  dataset = DatasetDict({
      'train': dataset['train'],
      'validation': dataset['valid'],
      'test': dataset['test'],
  })

  return dataset.map(lambda i: {'d': i['conversations'], 's': (i['situation'])})

dataset_params = {
    "knkarthick/dialogsum": ("d", "s"),
    "Estwld/empathetic_dialogues_llm": ("d", "s")
}

dataset_preprocesses = {
    'knkarthick/dialogsum': split_dialogue,
    "Estwld/empathetic_dialogues_llm": preprocess_empathetic_dialogues_llm,
}

dataset = NewDataset(dataset_params,
                     dataset_preprocesses=dataset_preprocesses,
                     target_col_name = 's',
                     input_col_name = 'd')
# for test run
# test_max_size = 10
# for split, ds in dataset.dict_dataset.items(): 
#     dataset.dict_dataset[split] = ds.select(range(10))
#

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

...

Estwld/empathetic_dialogues_llm have a custom fn


Map:   0%|          | 0/19533 [00:00<?, ? examples/s]

Map:   0%|          | 0/2770 [00:00<?, ? examples/s]

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['d', 's'],
    num_rows: 31993
}), 'validation': Dataset({
    features: ['d', 's'],
    num_rows: 3270
}), 'test': Dataset({
    features: ['d', 's'],
    num_rows: 4047
})}


In [6]:
def join_dialogue(d):
  s = ""

  for message in d:
    if message['role'] == 'user':
      s += f"You: {message['content']}\n"
    else:
      s += f"[CHARACTER]: {message['content']}\n"

  return s

def to_pygmalion(d, s, max_messages = 6):
  """
  Parameters:
    d: dialogue: massive of messages {content: <message text>, role: <user or assistant>}
    s: summary: the summarization of dialogue text

  Return the text relevant to Pygmalion dialogue format
  """
  text = f"*HISTORY: {s}*\n <START>\n"

  text += join_dialogue(d[-max_messages:])

  return text

# Add summaries with trained lora model

In [7]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import torch
from torch.nn import DataParallel

class SumModel():
  def __init__(cfg, model_preset, **generation_parameters)->None:
    # requires transformers and peft installed libs

    cfg.model_preset = model_preset
    cfg.generation_params = generation_parameters

    config = PeftConfig.from_pretrained(cfg.model_preset)

    model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
    cfg.tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

    cfg.lora_model = PeftModel.from_pretrained(model, cfg.model_preset)

    cfg.lora_model.print_trainable_parameters()

    cfg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    cfg.lora_model.model = DataParallel(cfg.lora_model.model)

  def __call__(self, text, **generation_params):
    tokens = self.tokenizer(text, return_tensors = 'pt', truncation=True, padding=True).to(self.device)
    if len(generation_params):
      gen = self.lora_model.generate(**tokens, **generation_params)
    else:
      gen = self.lora_model.generate(**tokens, **self.generation_params)

    return self.tokenizer.batch_decode(gen)

model = SumModel(model_preset = 'doublecringe123/bardt-large-cnn-dialoguesum-booksum-lora',
                 max_length = 96,
                 min_length = 26,
                 do_sample = True,
                 temperature = 0.9,
                 num_beams = 8,
                 repetition_penalty= 2.)
# Using Example:
# model("""Causal language modeling predicts the next token in a sequence of tokens, and the model can only attend to tokens on the left. This means the model cannot see future tokens. GPT-2 is an example of a causal language model.

# This guide will show you how to:

# Finetune DistilGPT2 on the r/askscience subset of the ELI5 dataset.
# Use your finetuned model for inference.""", min_length = 20, max_length=30)

adapter_config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

trainable params: 0 || all params: 408,059,904 || trainable%: 0.0


In [8]:
def to_pygmalion_with_autosummarization_batched(dialogues, max_messages=6):
    """
    Parameters:
      dialogues: List of dialogues, where each dialogue is a list of messages
      max_messages: Maximum number of messages to consider for history summarization

    Return the text relevant to Pygmalion dialogue format with automated history summarization
    """

    # Prepare the dialogues for batch processing
    processed_dialogues = [join_dialogue(dialogue[:-max_messages]) for dialogue in dialogues]

    # Perform batch summarization
    with torch.no_grad():
        summaries = model(processed_dialogues)

    text_list = []
    for dialogue, summary in zip(dialogues, summaries):
        text = f"*HISTORY: {summary}*\n<START>\n"
        text += join_dialogue(dialogue[-max_messages:])
        text_list.append(text)

    return text_list

# Build Finnal Dataset

In [9]:
from datasets import DatasetDict

dataset_d = DatasetDict(dataset.dict_dataset)
dataset_original_summaries = dataset_d.map(lambda x: {'d': to_pygmalion(x['d'], x['s'])}, remove_columns = ['d', 's'])

Map:   0%|          | 0/31993 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/4047 [00:00<?, ? examples/s]

In [10]:
dataset_generated_summaries = dataset_d.map(lambda x: {'d': to_pygmalion_with_autosummarization_batched(x['d'])},
                                           remove_columns=['d', 's'],
                                           batched=True,
                                           batch_size=8)

Map:   0%|          | 0/31993 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/4047 [00:00<?, ? examples/s]

In [11]:
from datasets import concatenate_datasets

# concatenate dataset dict (split, concanate, join to DictDataset)
dataset_finnal = DatasetDict({name1: concatenate_datasets([split1, split2])
 for (name1, split1), (name2, split2) in
  zip(dataset_original_summaries.items(), dataset_generated_summaries.items())})

In [12]:
dataset_finnal.save_to_disk(cfg.save_path)
dataset_finnal.push_to_hub(f"{cfg.username}/{cfg.save_path}", commit_message = 'first commit')

Saving the dataset (0/1 shards):   0%|          | 0/63986 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6540 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8094 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/64 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/471 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/doublecringe123/dataset_for_finetune_pygmalion/commit/e8d627237325cb10617732a02551aef65093feb7', commit_message='first commit', commit_description='', oid='e8d627237325cb10617732a02551aef65093feb7', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
from datasets import load_from_disk
# check up the dataset saved correctly
dataset_finnal1 = load_from_disk(cfg.save_path)
dataset_finnal1

DatasetDict({
    train: Dataset({
        features: ['d'],
        num_rows: 63986
    })
    validation: Dataset({
        features: ['d'],
        num_rows: 6540
    })
    test: Dataset({
        features: ['d'],
        num_rows: 8094
    })
})